In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Autism_spectrum_disorder_(ASD)"
cohort = "GSE87847"

# Input paths
in_trait_dir = "../../input/GEO/Autism_spectrum_disorder_(ASD)"
in_cohort_dir = "../../input/GEO/Autism_spectrum_disorder_(ASD)/GSE87847"

# Output paths
out_data_file = "../../output/preprocess/Autism_spectrum_disorder_(ASD)/GSE87847.csv"
out_gene_data_file = "../../output/preprocess/Autism_spectrum_disorder_(ASD)/gene_data/GSE87847.csv"
out_clinical_data_file = "../../output/preprocess/Autism_spectrum_disorder_(ASD)/clinical_data/GSE87847.csv"
json_path = "../../output/preprocess/Autism_spectrum_disorder_(ASD)/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"A Putative Blood-Based Biomarker for Autism Spectrum Disorder-Associated Ileocolitis"
!Series_summary	"Analysis of gene expression in inflamed gastrointestinal tissue and blood from GI-symptomatic children with ASD compared to non-inflamed tissue and blood from typically developing GI-syptomatic children. The hypothesis being tested was that peripheral blood would yield a surrogate biomarker for GI inflammation in children with ASD."
!Series_overall_design	"Total RNA was isolated from inflamed gastrointestinal tissue (terminal ileum and/or colon) and peripheral blood from children with ASD and corresponding (non-iflamed) tissue and blood from typically developing children."
Sample Characteristics Dictionary:
{0: ['sample type: Autism Spectrum Disorder', 'sample type: typically developing'], 1: ['disease state: inflamed', 'disease state: non-inflamed'], 2: ['Sex: male', 'Sex: female']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os
import json
from typing import Optional, Callable, Dict, Any

# 1. Determine if gene expression data is available
# From the background information, we can see this dataset contains gene expression data from tissues and blood
is_gene_available = True  # Gene expression data is available

# 2. Define variables for trait, age, and gender availability

# 2.1 Data Availability
# From the sample characteristics dictionary:
# Key 0 contains info about ASD vs typically developing (trait)
# Key 1 contains info about inflammation state (not our target trait)
# Key 2 contains gender information
# No age information is available

trait_row = 0  # ASD status is in row 0
age_row = None  # Age information is not available
gender_row = 2  # Gender information is in row 2

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """Convert ASD trait values to binary (1 for ASD, 0 for typically developing)"""
    if pd.isna(value):
        return None
    
    # Extract value after the colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if 'autism' in value.lower() or 'asd' in value.lower():
        return 1
    elif 'typically developing' in value.lower():
        return 0
    else:
        return None

def convert_age(value):
    """Convert age values to continuous numbers"""
    # We don't have age data in this dataset
    return None

def convert_gender(value):
    """Convert gender values to binary (0 for female, 1 for male)"""
    if pd.isna(value):
        return None
    
    # Extract value after the colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if 'male' in value.lower():
        return 1
    elif 'female' in value.lower():
        return 0
    else:
        return None

# 3. Save metadata - Initial filtering
# trait_row is not None, so trait data is available
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (if trait data is available)
if trait_row is not None:
    # Load the clinical data
    # Assuming clinical_data was obtained in a previous step and is available
    try:
        # This is a placeholder - the actual clinical_data should come from a previous step
        clinical_data = pd.read_csv(os.path.join(in_cohort_dir, "clinical_data.csv"))
        
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the extracted clinical features
        preview = preview_df(selected_clinical_df)
        print("Preview of selected clinical features:")
        print(preview)
        
        # Save the clinical data
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error in clinical feature extraction: {e}")
        print("Clinical data might not be available yet from previous steps.")


Error in clinical feature extraction: [Errno 2] No such file or directory: '../../input/GEO/Autism_spectrum_disorder_(ASD)/GSE87847/clinical_data.csv'
Clinical data might not be available yet from previous steps.


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['7A5', 'A1BG', 'A1CF', 'A26C3', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1',
       'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS', 'AACS', 'AADAC',
       'AADACL1', 'AADACL2', 'AADACL4', 'AADAT', 'AAGAB'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# I need to review the gene identifiers from the gene expression data

# These appear to be human gene symbols, not probe IDs or other identifiers that would require mapping.
# The list includes well-known gene symbols like A1BG, A2M, AAAS, AADAC, etc.
# These are standard HGNC gene symbols and don't require mapping to other identifiers.

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. We need to first create the selected_clinical_df using clinical_data from Step 1
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Preview the extracted clinical features
preview = preview_df(selected_clinical_df)
print("Preview of selected clinical features:")
print(preview)

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
# Create directory if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data with the 'geo_link_clinical_genetic_data' function from the library.
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)

# 3. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)

# 4. Determine whether the trait and some demographic features are severely biased, and remove biased features.
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information.
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data, 
    note="Dataset contains gene expression data from both blood and GI tissue of ASD and typically developing children."
)

# 6. If the linked data is usable, save it as a CSV file to 'out_data_file'.
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("The dataset was determined to be not usable for analysis.")

Preview of selected clinical features:
{'GSM2341810': [1.0, 1.0], 'GSM2341811': [1.0, 1.0], 'GSM2341812': [1.0, 1.0], 'GSM2341813': [1.0, 1.0], 'GSM2341814': [1.0, 1.0], 'GSM2341815': [1.0, 1.0], 'GSM2341816': [1.0, 1.0], 'GSM2341817': [1.0, 1.0], 'GSM2341818': [1.0, 1.0], 'GSM2341819': [1.0, 1.0], 'GSM2341820': [1.0, 1.0], 'GSM2341821': [1.0, 1.0], 'GSM2341822': [1.0, 1.0], 'GSM2341823': [1.0, 1.0], 'GSM2341824': [1.0, 1.0], 'GSM2341825': [1.0, 1.0], 'GSM2341826': [1.0, 1.0], 'GSM2341827': [1.0, 1.0], 'GSM2341828': [1.0, 1.0], 'GSM2341829': [1.0, 1.0], 'GSM2341830': [1.0, 1.0], 'GSM2341831': [1.0, 1.0], 'GSM2341832': [1.0, 1.0], 'GSM2341833': [1.0, 1.0], 'GSM2341834': [1.0, 1.0], 'GSM2341835': [1.0, 1.0], 'GSM2341836': [1.0, 1.0], 'GSM2341837': [1.0, 1.0], 'GSM2341838': [1.0, 1.0], 'GSM2341839': [1.0, 1.0], 'GSM2341840': [1.0, 1.0], 'GSM2341841': [1.0, 1.0], 'GSM2341842': [1.0, 1.0], 'GSM2341843': [1.0, 1.0], 'GSM2341844': [1.0, 1.0], 'GSM2341845': [1.0, 1.0], 'GSM2341846': [1.0, 1.0]

Normalized gene data saved to ../../output/preprocess/Autism_spectrum_disorder_(ASD)/gene_data/GSE87847.csv


For the feature 'Autism_spectrum_disorder_(ASD)', the least common label is '1.0' with 45 occurrences. This represents 48.39% of the dataset.
The distribution of the feature 'Autism_spectrum_disorder_(ASD)' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 93 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is severely biased.



Linked data saved to ../../output/preprocess/Autism_spectrum_disorder_(ASD)/GSE87847.csv
